# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

In [87]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt

tickers = ['AAPL', 'META', 'TSLA', 'NVDA', 'MSTR', 'PLTR']
data = yf.download(tickers, start='2024-06-01', end='2025-06-01')['Close']


[*********************100%***********************]  6 of 6 completed


In [88]:
returns = data.pct_change().dropna()

expected_returns = {stock: 252 * np.mean(returns[stock]) for stock in tickers}
covariance_matrix = returns.cov() * 252  
n_assets = len(tickers)
initial_weights = np.array([1 / n_assets] * n_assets)

constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    {'type': 'ineq', 'fun': lambda w: min(w) - 0.10},
    {'type': 'ineq', 'fun': lambda w: 0.35 - max(w)}
)


In [ ]:
# Low risk Function
def portfolio_volatility(weights):
    return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

result_low = minimize(portfolio_volatility, initial_weights, constraints=constraints)
low_risk_weights = result_low.x

volatilities = returns.std() * np.sqrt(252)
most_volatile_asset = volatilities.idxmax()

In [90]:
#High Risk Function
def negative_portfolio_volatility(weights):
    return -np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

result_max_vol = minimize(negative_portfolio_volatility, initial_weights, constraints=constraints)
max_vol_weights = result_max_vol.x

In [91]:
def portfolio_performance(weights):
    port_return = np.dot(weights, [expected_returns[t] for t in tickers])
    port_vol = portfolio_volatility(weights)
    return port_return, port_vol

low_ret, low_vol = portfolio_performance(low_risk_weights)
max_vol_ret, max_vol_std = portfolio_performance(max_vol_weights)

In [92]:
print("\nLow-Risk Portfolio")
for t, w in zip(tickers, low_risk_weights):
    print(f"{t}: {w:.4f}")
print(f"Expected Return: {low_ret:.2%}, Volatility: {low_vol:.2%}")


print("\nHigh-Risk Portfolio")
for t, w in zip(tickers, max_vol_weights):
    print(f"{t}: {w:.4f}")
print(f"Expected Return: {max_vol_ret:.2%}, Volatility: {max_vol_std:.2%}")



Low-Risk Portfolio
AAPL: 0.3500
META: 0.2500
TSLA: 0.1000
NVDA: 0.1000
MSTR: 0.1000
PLTR: 0.1000
Expected Return: 59.83%, Volatility: 37.82%

High-Risk Portfolio
AAPL: 0.1000
META: 0.1000
TSLA: 0.3500
NVDA: 0.1002
MSTR: 0.1002
PLTR: 0.2497
Expected Return: 107.06%, Volatility: 56.64%


### Low-Risk Portfolio: 
### More established companies like Meta and Apple were emphasized in this portfolio, which has historically lower volatility compared to the other assets. The remaining assetrs are held at their minimum allowable weight to limit exposure to the higher risk.
### Therefore, the portfolio aims to balance growth potential with stability by concetrating on mega-cap tech firms. The lower volatility, 37.82% , means annual portfolio value changes are expected to be smaller compared a to higher-risk setup. With this lower volatility, there is similarly a tradeoff with expected returns, where expected gains and losses would be lower.



### High-Risk Portfolio:
### This portfolio maximizes exposure to stocks with historically higher price volatility, namely Tesla and Palantir. Designed for investors seeking higher potential returns, accepting significantly greater risk in terms of price fluctuations. 
### The higher volatility of 56.64% indicates a larger range of possible outcomes, both gains and losses, in any given period. It reflects a high-risk, high-reward investment philosophy focused on aggressive growth.
